In [1]:
import sqlite3 as sql
import pandas as pd

In [2]:
class sqlite_con_queries_inserts:
    # Cursor is now created with class
    def __init__(self, path, queries):
        self.path = path
        self.connection = self.create_connection()
        self.cursor = self.connection.cursor()
        self.queries = queries

    # Connection function
    def create_connection(self):
        connection = None
        try:
            connection = sql.connect(self.path)
            print("Connection to SQLite DB successful")
        except sql.Error as e:
            print(f"The error '{e}' occurred")
        return connection

    # Query function inputting cursor with a list of queries.
    def execute_query(self, data=None, print_message=True):
        try:
            for query in self.queries:
                if data:
                    self.cursor.execute(query, data)
                else:
                    self.cursor.execute(query)
            # Commit only if all statements succeeded
            self.cursor.connection.commit()
            if print_message:
                print("Queries executed successfully")
        except sql.Error as e:
            print("An error occurred:", e)
            # Rollback the transaction in case of error
            self.cursor.connection.rollback()
    
    # Inerts static function
    @staticmethod
    def execute_inserts(table, query, path):
        # Convert DataFrame rows to list of dictionaries
        my_list = table.to_dict(orient='records')
        # Iterate through the list of dictionaries and insert data
        for row in my_list:
            try:
                sqlite_con_queries_inserts(path, query).execute_query(data=row, print_message=False)
            except sql.Error as e:
                print("An error occurred:", e)
        print("Insert query executed successfully")

# Inserting Initial 6 Tables

In [ ]:
# CREATE TABLE queries in related order
create_table_queries = [
"""
CREATE TABLE IF NOT EXISTS airliners (
  airline_id VARCHAR PRIMARY KEY,
  name VARCHAR,
  location_country VARCHAR,
  location_region VARCHAR,
  location_city VARCHAR
);
""",
"""
CREATE TABLE IF NOT EXISTS suppliers (
  company_id VARCHAR PRIMARY KEY,
  name VARCHAR NOT NULL,
  location_country VARCHAR,
  location_region VARCHAR,
  location_city VARCHAR,
  global_ife_share_billions_2022 FLOAT,
  revenue_billions_2019 FLOAT,
  revenue_estimate_billions FLOAT,
  strengths VARCHAR,
  weaknesses VARCHAR,
  opportunities VARCHAR,
  threats VARCHAR
);
""",
"""
CREATE TABLE IF NOT EXISTS software_supplier (
  software_id VARCHAR PRIMARY KEY,
  company_id VARCHAR,
  co_company_id VARCHAR,
  apps_features VARCHAR,
  FOREIGN KEY (company_id) REFERENCES suppliers(company_id)
);
""",
"""
CREATE TABLE IF NOT EXISTS hardware_supplier (
  hardware_id VARCHAR PRIMARY KEY,
  company_id VARCHAR,
  co_company_id VARCHAR,
  about VARCHAR,
  FOREIGN KEY (company_id) REFERENCES suppliers(company_id)
);
""",
"""
CREATE TABLE IF NOT EXISTS airline_software_relation (
  relation_id VARCHAR PRIMARY KEY,
  airline_id VARCHAR,
  software_id VARCHAR,
  relation_description VARCHAR,
  date TIMESTAMP,
  FOREIGN KEY (airline_id) REFERENCES airliners(airline_id),
  FOREIGN KEY (software_id) REFERENCES software(software_id)
);
""",
"""
CREATE TABLE IF NOT EXISTS airline_hardware_relation (
  relation_id VARCHAR PRIMARY KEY,
  airline_id VARCHAR,
  hardware_id VARCHAR,
  relation_description VARCHAR,
  date TIMESTAMP,
  FOREIGN KEY (airline_id) REFERENCES airliners(airline_id),
  FOREIGN KEY (hardware_id) REFERENCES hardware(hardware_id)
);
"""
]
path = 'TRL1_Database.sqlite'
queries = create_table_queries
# Call the function to create the tables
sqlite_con_queries_inserts(path, queries).execute_query()

In [ ]:
# To remove the table, you have to:
#     Disable foreign key constraints.
#     Drop the table.
#     Update the FKs in the connecting tables to NULL values.
#     Enable the foreign key constraints.

# See the following statements:

"""
PRAGMA foreign_keys = OFF;

DROP TABLE table_name;

UPDATE connecting_table
SET foreign_key_name = NULL;

PRAGMA foreign_keys = ON;
"""

# Inserting Airliners Table

In [ ]:
airliner_db = pd.read_excel('QIS_TRL1_DB.xlsx', sheet_name='Airliners', index_col=None)
airliner_db = airliner_db.rename(columns={"Airliner ID": "airline_id", 
                            "Name": "name", 
                            "Location (Country)": "location_country",
                            "Location (Province/State/Region)": "location_region",
                            "Location (City)": "location_city"})
airliner_db

In [ ]:
table = airliner_db
# The column names in the airliner_db are renamed to match the tables in SQLite.
insert_stations = [
"""
INSERT INTO
  airliners (airline_id, name, location_country, location_region, location_city)
VALUES
  (:airline_id, :name, :location_country, :location_region, :location_city);
"""
]
query = insert_stations

path = 'TRL1_Database.sqlite'
# Execute function to insert values from Pandas database into SQLite.
sqlite_con_queries_inserts.execute_inserts(table, query, path)

# Inserting Supplier Company Table

In [ ]:
airliner_db = pd.read_excel('QIS_TRL1_DB.xlsx', sheet_name='Supplier Company', index_col=None)
airliner_db = airliner_db.rename(columns={"Company ID": "company_id", 
                            "Name": "name", 
                            "Location (Country)": "location_country",
                            "Location (Province/State/Region)": "location_region",
                            "Location (City)": "location_city",
                            "Global Share in IFE & Connectivity Market 2022 (billion USD)": "global_ife_share_billions_2022",
                            "Overall Revenue 2019 (billion USD)": "revenue_billions_2019",
                            "Overall Annual Revenue (Rough Estimate) (billion USD)": "revenue_estimate_billions",
                            "Strengths": "strengths",
                            "Weaknesses": "weaknesses", 
                            "Opportunities": "opportunities",
                            "Threats": "threats"})
airliner_db

In [ ]:
table = airliner_db
# The column names in the airliner_db are renamed to match the tables in SQLite.
insert_stations = [
"""
INSERT INTO
  suppliers (company_id, name, location_country, location_region, location_city, global_ife_share_billions_2022, revenue_billions_2019, revenue_estimate_billions, strengths, weaknesses, opportunities, threats)
VALUES
  (:company_id, name, :location_country, :location_region, :location_city, :global_ife_share_billions_2022, :revenue_billions_2019, :revenue_estimate_billions, :strengths, :weaknesses, :opportunities, :threats);
"""
]
query = insert_stations

path = 'TRL1_Database.sqlite'
# Execute function to insert values from Pandas database into SQLite.
sqlite_con_queries_inserts.execute_inserts(table, query, path)

# Inserting the Company Supplying Software Table

In [ ]:
airliner_db = pd.read_excel('QIS_TRL1_DB.xlsx', sheet_name='Company Supplying Software', index_col=None)
airliner_db = airliner_db.rename(columns={"Software ID": "software_id",
                            "Company ID": "company_id",
                            "Co-Company ID (if applicable)": "co_company_id",
                            "Apps/Features": "apps_features"})
airliner_db

In [ ]:
table = airliner_db
# The column names in the airliner_db are renamed to match the tables in SQLite.
insert_stations = [
"""
INSERT INTO
  software_supplier (software_id, company_id, co_company_id, apps_features)
VALUES
  (:software_id, :company_id, :co_company_id, :apps_features);
"""
]
query = insert_stations

path = 'TRL1_Database.sqlite'
# Execute function to insert values from Pandas database into SQLite.
sqlite_con_queries_inserts.execute_inserts(table, query, path)

# Inserting the Airline Software Relation Table

In [ ]:
airliner_db = pd.read_excel('QIS_TRL1_DB.xlsx', sheet_name='Airline Software Relation', index_col=None)
airliner_db = airliner_db.rename(columns={"Relation ID": "relation_id",
                            "Airliner ID": "airline_id",
                            "Software ID": "software_id",
                            "Relation Description": "relation_description", 
                            "Date": "date"})
airliner_db["date"] = pd.to_datetime(airliner_db["date"], format='%Y-%m-%d', errors='coerce')
# If cannont import the date, convert it to a string
airliner_db["date"] = airliner_db["date"].dt.date
print(airliner_db.dtypes)
airliner_db

In [ ]:
table = airliner_db
# The column names in the airliner_db are renamed to match the tables in SQLite.
insert_stations = [
"""
INSERT INTO
  airline_software_relation (relation_id, airline_id, software_id, relation_description, date)
VALUES
  (:relation_id, :airline_id, :software_id, :relation_description, :date);
"""
]
query = insert_stations

path = 'TRL1_Database.sqlite'
# Execute function to insert values from Pandas database into SQLite.
sqlite_con_queries_inserts.execute_inserts(table, query, path)

# Inserting the Company Supplying Hardware Table

In [ ]:
airliner_db = pd.read_excel('QIS_TRL1_DB.xlsx', sheet_name='Company Supplying Hardware', index_col=None)
airliner_db = airliner_db.rename(columns={"Hardware ID": "hardware_id",
                            "Company ID": "company_id",
                            "Co-Company ID (if applicable)": "co_company_id",
                            "About": "about"})
airliner_db

In [ ]:
table = airliner_db
# The column names in the airliner_db are renamed to match the tables in SQLite.
insert_stations = [
"""
INSERT INTO
  hardware_supplier (hardware_id, company_id, co_company_id, about)
VALUES
  (:hardware_id, :company_id, :co_company_id, :about);
"""
]
query = insert_stations

path = 'TRL1_Database.sqlite'
# Execute function to insert values from Pandas database into SQLite.
sqlite_con_queries_inserts.execute_inserts(table, query, path)

# Inserting the Airline Hardware Relation Table

In [ ]:
airliner_db = pd.read_excel('QIS_TRL1_DB.xlsx', sheet_name='Airline Hardware Relation', index_col=None)
airliner_db = airliner_db.rename(columns={"Relation ID": "relation_id",
                            "Airliner ID": "airline_id",
                            "Hardware ID": "hardware_id",
                            "Relation Description": "relation_description", 
                            "Date": "date"})
airliner_db["date"] = pd.to_datetime(airliner_db["date"], format='%Y-%m-%d', errors='coerce')
# If cannont import the date, convert it to a string
airliner_db["date"] = airliner_db["date"].dt.strftime('% d % m % Y')
print(airliner_db.dtypes)
airliner_db

In [ ]:
table = airliner_db
# The column names in the airliner_db are renamed to match the tables in SQLite.
insert_stations = [
"""
INSERT INTO
  airline_hardware_relation (relation_id, airline_id, hardware_id, relation_description, date)
VALUES
  (:relation_id, :airline_id, :hardware_id, :relation_description, :date);
"""
]
query = insert_stations

path = 'TRL1_Database.sqlite'
# Execute function to insert values from Pandas database into SQLite.
sqlite_con_queries_inserts.execute_inserts(table, query, path)

# Inserting the Airline Aircraft Entertainment Score Table

In [ ]:
# CREATE TABLE query
create_table_queries = [
"""
CREATE TABLE IF NOT EXISTS aircraft_and_entertainment (
  index_id VARCHAR PRIMARY KEY,
  airline_id VARCHAR,
  airline_name VARCHAR,
  aircraft VARCHAR,
  ent_score_count FLOAT,
  ent_score_mean FLOAT,
  ent_score_std FLOAT,
  ent_score_min FLOAT,
  ent_score_q1 FLOAT,
  ent_score_med FLOAT,
  ent_score_q3 FLOAT,
  ent_score_max FLOAT,
  FOREIGN KEY (airline_id) REFERENCES airliners(airline_id)
);
"""
]
path = 'TRL1_Database.sqlite'
queries = create_table_queries
# Call the function to create the tables
sqlite_con_queries_inserts(path, queries).execute_query()

In [ ]:
airliner_db = pd.read_excel('QIS_TRL1_DB.xlsx', sheet_name='Airline Aircraft Ent. Score', index_col=None)
airliner_db = airliner_db.rename(columns={"ent_score_25%": "ent_score_q1",
                            "ent_score_50%": "ent_score_med",
                            "ent_score_75%": "ent_score_q3"})
airliner_db

In [ ]:
table = airliner_db
# The column names in the airliner_db are renamed to match the tables in SQLite.
insert_stations = [
"""
INSERT INTO
  aircraft_and_entertainment (index_id, airline_id, airline_name, aircraft, ent_score_count, ent_score_mean, ent_score_std, ent_score_min, ent_score_q1, ent_score_med, ent_score_q3, ent_score_max)
VALUES
  (:index_id, :airline_id, :airline_name, :aircraft, :ent_score_count, :ent_score_mean, :ent_score_std, :ent_score_min, :ent_score_q1, :ent_score_med, :ent_score_q3, :ent_score_max);
"""
]
query = insert_stations

path = 'TRL1_Database.sqlite'
# Execute function to insert values from Pandas database into SQLite.
sqlite_con_queries_inserts.execute_inserts(table, query, path)

# Inserting Competitors Table

In [ ]:
# CREATE TABLE query
create_table_queries = [
"""
CREATE TABLE IF NOT EXISTS competitors (
  competition_id VARCHAR PRIMARY KEY,
  primary_company_id VARCHAR,
  competitor_id VARCHAR,
  FOREIGN KEY (primary_company_id) REFERENCES suppliers(company_id),
  FOREIGN KEY (competitor_id) REFERENCES suppliers(company_id)
);
"""
]
path = 'TRL1_Database.sqlite'
queries = create_table_queries
# Call the function to create the tables
sqlite_con_queries_inserts(path, queries).execute_query()

In [ ]:
airliner_db = pd.read_excel('QIS_TRL1_DB.xlsx', sheet_name='Competitors', index_col=None)
airliner_db = airliner_db.rename(columns={"Competition ID": "competition_id", 
                            "Primary Company ID": "primary_company_id", 
                            "Competitor ID": "competitor_id"})
airliner_db = airliner_db[['competition_id', 'primary_company_id', 'competitor_id']]
airliner_db

In [ ]:
table = airliner_db
# The column names in the airliner_db are renamed to match the tables in SQLite.
insert_stations = [
"""
INSERT INTO
  competitors (competition_id, primary_company_id, competitor_id)
VALUES
  (:competition_id, :primary_company_id, :competitor_id);
"""
]
query = insert_stations

path = 'TRL1_Database.sqlite'
# Execute function to insert values from Pandas database into SQLite.
sqlite_con_queries_inserts.execute_inserts(table, query, path)

# Inserting Wifi Availability Table

In [ ]:
# CREATE TABLE query
create_table_queries = [
"""
CREATE TABLE IF NOT EXISTS wifi_availability (
  wifi_id VARCHAR PRIMARY KEY,
  airline_id VARCHAR,
  payment_plan VARCHAR,
  aircraft_availability VARCHAR,
  minimum_price_usd FLOAT,
  maximum_price_usd FLOAT,
  expenditure_by_airline_billion_usd FLOAT,
  FOREIGN KEY (airline_id) REFERENCES airliners(airline_id)
);
"""
]
path = 'TRL1_Database.sqlite'
queries = create_table_queries
# Call the function to create the tables
sqlite_con_queries_inserts(path, queries).execute_query()

In [ ]:
airliner_db = pd.read_excel('QIS_TRL1_DB.xlsx', sheet_name='Wifi Availability', index_col=None)
airliner_db = airliner_db.rename(columns={"Wifi ID": "wifi_id", 
                            "Airline ID": "airline_id",
                            "Payment Plan": "payment_plan",
                            "Aircraft Availability": "aircraft_availability",
                            "Minimum Price (USD)": "minimum_price_usd",
                            "Maximum Price (USD)": "maximum_price_usd",
                            "Expenditure by Airline (billion USD)": "expenditure_by_airline_billion_usd"})
airliner_db = airliner_db[['competition_id', 'primary_company_id', 'competitor_id']]
airliner_db

In [ ]:
table = airliner_db
# The column names in the airliner_db are renamed to match the tables in SQLite.
insert_stations = [
"""
INSERT INTO
  competitors (wifi_id, airline_id, payment_plan, aircraft_availability, minimum_price_usd, maximum_price_usd, expenditure_by_airline_billion_usd)
VALUES
  (:wifi_id, :airline_id, :payment_plan, :aircraft_availability, :minimum_price_usd, :maximum_price_usd, :expenditure_by_airline_billion_usd);
"""
]
query = insert_stations

path = 'TRL1_Database.sqlite'
# Execute function to insert values from Pandas database into SQLite.
sqlite_con_queries_inserts.execute_inserts(table, query, path)